## Presence of tfs family in different cell line

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from comut import comut
import seaborn as sns

In [ ]:
plt.rcParams['figure.dpi'] = 150

In [ ]:
safari_options = Options()
safari_options.add_argument("headless")

In [ ]:
baseurl = r'https://www.encodeproject.org/matrix/?type=Experiment&control_type%21=%2A&status=released&perturbed=false&assay_title=TF+ChIP-seq&target.investigated_as=transcription+factor&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&biosample_ontology.classification=cell+line&biosample_ontology.term_name='

In [ ]:
# select cell : HepG2, K562, HEK293, GM12878, MCF-7, WTC11, A549, SK-N-SH, HeLa-S3, H1

In [ ]:
cell_lines = []
tfs = []
counts = []

for cellline in ['HepG2', 'K562', 'HEK293', 'GM12878', 'MCF-7', 'WTC11', 'A549', 'SK-N-SH', 'HeLa-S3', 'H1']:
    url = baseurl + cellline
    dr = webdriver.Safari(options=safari_options)
    dr.implicitly_wait(10)
    dr.get(url)
    dr.implicitly_wait(10)
    tfs_div = dr.find_element(By.XPATH,'//*[@id="facet-group-Assay-Experiment"]/div[4]/div/div[2]/ul/div[1]')
    tf_div = tfs_div.find_elements(By.CLASS_NAME, 'facet-term')
    for index, tf in enumerate(tf_div):
        cell_lines.append(cellline)
        tfs.append(tf.find_element(By.CLASS_NAME, 'facet-term__text').text)
        counts.append(tf.find_element(By.CLASS_NAME, 'facet-term__count').text)
    dr.quit()


In [ ]:
tfs_data_encode = pd.DataFrame({
    "Cell" : cell_lines,
    "TF" : tfs,
    "Count" : counts
})

In [ ]:
tfs_data_encode.groupby('Cell').size()

In [ ]:
tfs_family = pd.read_csv('data/tfs_family.csv')
tfs_family_dict = dict(tfs_family[['Name','Family']].values)

# add Unkonw
for i in tfs_data_encode['TF']:
    if i in tfs_family_dict.keys():
        pass
    else:
        tfs_family_dict[i] = "Unknown"



In [ ]:
tfs_data_encode['Family'] = [ tfs_family_dict[i] for i in tfs_data_encode['TF'] ]

In [ ]:
sns.barplot(tfs_data_encode.groupby(['Cell', 'Family']).size().reset_index(name="Count"), x = "Family", y="Count", hue="Cell")
plt.tick_params(axis='x', labelrotation=45)
plt.show()

In [ ]:
tfs_data_encode.groupby(['Cell', 'Family']).size()

In [ ]:
len(tfs_data_encode.drop_duplicates('Family')['Family'])

In [ ]:
comut_data = tfs_data_encode.groupby(['Cell', 'Family']).size().reset_index(name="Count")
comut_data.columns = ['category', 'sample', 'value']
#comut_data['value'] = [ 1/ (1 + np.exp(-i)) for i in comut_data['value'].tolist() ]
comut_data['value'] = "1"

In [ ]:
family_comut = comut.CoMut()
mut_mapping={"1":"#4876FF"}
family_comut.add_categorical_data(comut_data, name='type',mapping=mut_mapping)
#family_comut.add_continuous_data(comut_data, name='type')
family_comut.plot_comut(figsize = (10,3),x_padding = 0.04, y_padding = 0.04)
plt.savefig("results/encode.tf.family.pdf")

## Spider - Encode Project for each cell line

In [ ]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

In [ ]:
#encode_hg19 = pd.read_csv('/Users/cmf/Downloads/TFNet-single-tf/data/encode/encode.report.select.cell.tfs.hg19.tsv',header=0, sep='\t',skiprows=1)
encode_hg19 = pd.read_csv('/Users/cmf/Downloads/TFNet-single-tf/data/encode/encode.report.select.cell.tfs.hg19.tsv',header=0, sep='\t')
encode_grch38 = pd.read_csv('/Users/cmf/Downloads/TFNet-single-tf/data/encode/encode.report.select.cell.tfs.grch38.tsv',header=0, sep='\t')

In [ ]:
encode_hg19[['Accession', 'Target gene symbol', 'Biosample term name']]
base_url = 'https://www.encodeproject.org/experiments/'

In [ ]:
safari_options = Options()
safari_options.add_argument("headless")

In [ ]:
projects = []
file_accessions= []
file_types = []
output_types = []
isogenic_replicates = []
genomes = []
encode_pipes = []

dr = webdriver.Safari(options=safari_options)
dr.implicitly_wait(10)

#for accession in encode_hg19['Accession']:
for accession in encode_hg19['Accession'].tolist()[728:]:
    url = base_url + accession
    dr.get(url)
    sleep(3)
    dr.find_element(By.XPATH,'//*[@id="content"]/div/div[3]/div[2]/div[2]/div[1]/ul/li[3]/a').send_keys(Keys.ENTER)
    try:
        WebDriverWait(dr,10,0.2).until(EC.presence_of_element_located((By.XPATH,'//*[@id="tables"]/div/div[2]/div[2]/div')))
    except TimeoutException:
        sleep(3)
    encode_div = dr.find_element(By.XPATH, '//*[@id="tables"]/div/div[2]/div[2]/div')
    for index, table in enumerate(encode_div.find_elements(By.CLASS_NAME,'table__sortable')):
        if index != 0:
            if table.find_element(By.CLASS_NAME, 'title-main-content__analysis-title').text.find("hg19") !=-1 :
                encode_pipe = table.find_element(By.CLASS_NAME, 'title-main-content__analysis-title').text.split(" ")[0]
                tbody = table.find_element(By.TAG_NAME,'tbody')
                for tr in tbody.find_elements(By.TAG_NAME,'tr'):
                    tds = tr.find_elements(By.TAG_NAME,'td')
                    if tds[2].text == 'bed narrowPeak' and tds[3].text == 'optimal IDR thresholded peaks':
                        projects.append(accession)
                        genomes.append("hg19")
                        file_accessions.append(tds[0].text.rstrip("Open file informationDownload"))
                        file_types.append(tds[2].text)
                        output_types.append(tds[3].text)
                        isogenic_replicates.append(tds[4].text)
                        encode_pipes.append(encode_pipe)
                        print(accession)
    dr.quit()


encode_hg19_spider = pd.DataFrame({
    "project" : projects,
    "genome" : genomes,
    "file_accession" : file_accessions,
    "type" : file_types,
    "output_type" : output_types,
    "replicates" : isogenic_replicates,
    "pipe": encode_pipes
})

encode_hg19_spider.to_csv("data/encode_hg19_spider.csv",header=True, index=False)




In [ ]:
projects = []
file_accessions= []
file_types = []
output_types = []
isogenic_replicates = []
genomes = []
encode_pipes = []

dr = webdriver.Safari(options=safari_options)
dr.implicitly_wait(10)

for accession in encode_grch38['Accession']:
    url = base_url + accession
    dr.get(url)
    sleep(3)
    dr.find_element(By.XPATH,'//*[@id="content"]/div/div[3]/div[2]/div[2]/div[1]/ul/li[3]/a').send_keys(Keys.ENTER)
    try:
        WebDriverWait(dr,10,0.2).until(EC.presence_of_element_located((By.XPATH,'//*[@id="tables"]/div/div[2]/div[2]/div')))
    except TimeoutException:
        sleep(3)
    encode_div = dr.find_element(By.XPATH, '//*[@id="tables"]/div/div[2]/div[2]/div')
    for index, table in enumerate(encode_div.find_elements(By.CLASS_NAME,'table__sortable')):
        if index != 0:
            if table.find_element(By.CLASS_NAME, 'title-main-content__analysis-title').text.find("GRCh38") !=-1 :
                encode_pipe = table.find_element(By.CLASS_NAME, 'title-main-content__analysis-title').text.split(" ")[0]
                tbody = table.find_element(By.TAG_NAME,'tbody')
                for tr in tbody.find_elements(By.TAG_NAME,'tr'):
                    tds = tr.find_elements(By.TAG_NAME,'td')
                    if tds[2].text == 'bed narrowPeak' and (tds[3].text == 'optimal IDR thresholded peaks' or tds[3].text == 'IDR thresholded peaks'):
                        projects.append(accession)
                        print(accession)
                        genomes.append("GRCh38")
                        file_accessions.append(tds[0].text.rstrip("Open file informationDownload"))
                        file_types.append(tds[2].text)
                        output_types.append(tds[3].text)
                        isogenic_replicates.append(tds[4].text)
                        encode_pipes.append(encode_pipe)
    dr.quit()


encode_grch38_spider = pd.DataFrame({
    "project" : projects,
    "genome" : genomes,
    "file_accession" : file_accessions,
    "type" : file_types,
    "output_type" : output_types,
    "replicates" : isogenic_replicates,
    "pipe": encode_pipes
})

encode_grch38_spider.to_csv("data/encode_grch38_spider.csv",header=True, index=False)

In [ ]:
df_nation_freq_all = pd.read_csv('/Users/cmf/Downloads/mhc-related-backup/filter_mhc_ratio_nation.csv')

In [ ]:
df_nation_freq_all[df_nation_freq_all['nation'] == 'Western Asia'].sort_values(by="Ratio",ascending=False)[['nation','Allele','Ratio']].to_csv("data/WesternAsia_MHC_Ratio.csv", header=True, index=False)